In [ ]:
import sqlite3
import os
import csv
import pandas as pd

In [ ]:
# Function to create a database and table using column names from the CSV file
def create_database_from_csv(csv_filename, database_name):
    conn = sqlite3.connect(database_name)
    cursor = conn.cursor()

    with open(csv_filename, 'r') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)  # Read the header row to get column names

        # Generate the CREATE TABLE statement using column names from the CSV file
        create_table_sql = f"CREATE TABLE IF NOT EXISTS pointx_fbs_txn_rpt_dly ({', '.join([f'{col} TEXT' for col in header])})"

        cursor.execute(create_table_sql)

        # Read the data from the CSV file and insert it into the table
        for row in csv_reader:
            insert_sql = f"INSERT INTO pointx_fbs_txn_rpt_dly ({', '.join(header)}) VALUES ({', '.join(['?']*len(header))})"
            cursor.execute(insert_sql, row)

    conn.commit()
    conn.close()


In [ ]:
# Specify the CSV file and database name
csv_filename = 'pointx_fbs_rpt_dly.csv'
database_name = 'PointX.sqlite'

csv_path = os.path.join("src",csv_filename)
db_path = os.path.join("db",database_name)

# Create the database and table using column names from the CSV file

# create_database_from_csv(csv_path, db_path)

In [ ]:
def query_database(query, database_name="db/PointX.sqlite"):
    conn = sqlite3.connect(database_name)
    cursor = conn.cursor()

    cursor.execute(query)
    results = cursor.fetchall()
    # try:
    #     cursor.execute(query)
    #     results = cursor.fetchall()
    # except:
    #     results = "Error fetching data"

    conn.close()
    return results

In [ ]:
df = pd.read_excel('src/GPT_output.xlsx')
selected_columns = ['Question','SQL','pred_sql']
excluded_chars = ['`']

compare_result_csv = 'src/compare_result.csv'
label_result = []
pred_result = []

for index, row in df[selected_columns].iterrows():
    question = row['Question']
    pred_sql = ''.join(char for char in row['pred_sql'] if char not in excluded_chars)

    #Change table name in label to match the table name in the database
    label_sql = ''.join(char for char in row['SQL'] if char not in excluded_chars)
    label_sql = label_sql.replace('team_tds.tds_intern.pointx_fbs_txn_rpt_dly','pointx_fbs_txn_rpt_dly')
    
    label_result.append(query_database(label_sql))
    pred_result.append(query_database(pred_sql))

df['label_result'] = label_result
df['pred_result'] = pred_result

In [ ]:
df.to_csv(compare_result_csv, index=False)

In [ ]:
print(' Question:',question,'\n\n','Label SQL:',label_sql,'\n\n','Predicted SQL:',pred_sql,'\n\n')

In [ ]:
query_database(query)

In [ ]:
query_database(pred_sql)